In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras import layers, models
from sklearn.metrics import classification_report

2023-09-24 17:42:01.579897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def get_data():
    train_data = "Data/train"
    val_data = "Data/valid"
    return train_data, val_data

In [3]:
image_size = 224
batch_size = 50
def get_preprocessed_data(train_data, val_data):
    image_size = 224
    batch_size = 20
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255,
        shear_range= 0.2,
        horizontal_flip = True
    )
    test_val_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )
    train_generator = train_datagen.flow_from_directory(
        directory = train_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=True
    )
    val_generator = test_val_datagen.flow_from_directory(
        directory = val_data,
        target_size=(image_size, image_size),
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=False
    )
    return train_generator, val_generator   

In [4]:
train_data, val_data = get_data()
train_ds, val_ds = get_preprocessed_data(train_data, val_data)

Found 10555 images belonging to 4 classes.
Found 2790 images belonging to 4 classes.


In [5]:
net = keras.applications.mobilenet_v2.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=(image_size, image_size, 3)
    )
net.summary()   
for layer in net.layers[:-2]:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(net.output)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
prediction = layers.Dense(4, activation="softmax")(x)
model = models.Model(inputs = net.input, outputs = prediction)
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [6]:
model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(learning_rate=0.001),
        metrics = ['acc']
)

In [7]:
history = model.fit(
        train_ds,
        epochs = 8,
        validation_data=val_ds
)

Epoch 1/8
528/528 [==============================] - 415s 778ms/step - loss: 0.2838 - acc: 0.8947 - val_loss: 0.0891 - val_acc: 0.9649
Epoch 2/8
528/528 [==============================] - 419s 793ms/step - loss: 0.1853 - acc: 0.9308 - val_loss: 0.0779 - val_acc: 0.9731
Epoch 3/8
528/528 [==============================] - 442s 837ms/step - loss: 0.1600 - acc: 0.9377 - val_loss: 0.0791 - val_acc: 0.9720
Epoch 4/8
528/528 [==============================] - 410s 777ms/step - loss: 0.1407 - acc: 0.9462 - val_loss: 0.0634 - val_acc: 0.9749
Epoch 5/8
528/528 [==============================] - 411s 778ms/step - loss: 0.1269 - acc: 0.9526 - val_loss: 0.0643 - val_acc: 0.9778
Epoch 6/8
528/528 [==============================] - 412s 780ms/step - loss: 0.1088 - acc: 0.9590 - val_loss: 0.0617 - val_acc: 0.9785
Epoch 7/8
528/528 [==============================] - 413s 782ms/step - loss: 0.1105 - acc: 0.9587 - val_loss: 0.0511 - val_acc: 0.9824
Epoch 8/8
528/528 [==============================] - 41

In [8]:
scores = model.evaluate(val_ds, verbose=1)
print(f"Loss: {scores[0]}, Accuracy : {scores[1]}")

140/140 [==============================] - 78s 560ms/step - loss: 0.0844 - acc: 0.9649
Loss: 0.08435304462909698, Accuracy : 0.9648745656013489


In [9]:
test_pred = model.predict(val_ds, verbose = 1)
test_labels = np.argmax(test_pred, axis=1)

140/140 [==============================] - 80s 568ms/step


In [10]:
class_labels = val_ds.class_indices
class_labels = {v:k for k,v in class_labels.items()}
classes = list(class_labels.values())
print(classes)
print('Classification Report')
print(classification_report(val_ds.classes, test_labels, target_names=classes))

['Corn Common rust', 'Corn Gray leaf spot', 'Corn Healthy', 'Corn Northern Leaf Blight']
Classification Report
                           precision    recall  f1-score   support

         Corn Common rust       1.00      1.00      1.00       740
      Corn Gray leaf spot       0.86      0.99      0.92       560
             Corn Healthy       1.00      1.00      1.00       740
Corn Northern Leaf Blight       0.99      0.88      0.93       750

                 accuracy                           0.96      2790
                macro avg       0.96      0.97      0.96      2790
             weighted avg       0.97      0.96      0.97      2790



In [11]:
model.save("model/CornLeafMobileNet.h5")

/Users/dipit/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
